# WGS Analysis of Mbovis Samples from Algeria

## Jennifer J. Stiens
### j.j.stiens@gmail.com

### Project start: 8 December 2022

First step is variant calling in these samples vs reference strain. Secondly, identifying a clade? How do these strains fit in the phylogeny of Mbovis? 

Are these different samples of one strain? Taxon ID '233413' which is Mbovis AF2122/97, or thought to be different strains?


## References

[Zimpel et al, 2020](https://www.frontiersin.org/article/10.3389/fmicb.2020.00843)

[Loiseau et al, 2020](https://pubmed.ncbi.nlm.nih.gov/32211193/)

[Zimpel et al, 2017](https://www.frontiersin.org/articles/10.3389/fmicb.2017.02389/full)

[Reis and Cunha, 2021](https://doi.org/10.1038/s41598-021-98226-y)

[Branger et al, 2020](https://pubmed.ncbi.nlm.nih.gov/32240800/)

[Zwyer et al, 2021](https://s3-eu-west-1.amazonaws.com/openreseurope/manuscripts/15458/2094db28-ac1b-41b6-9bca-bd3e5e6b28ad_14029_-_daniela_brites_v2.pdf?doi=10.12688/openreseurope.14029.2&numberOfBrowsableCollections=88&numberOfBrowsableInstitutionalCollections=0&numberOfBrowsableGateways=4)




## Pipelines

Refer to Zimpel pipeline: 
[Zimpel snp pipeline](https://github.com/LaPAM-USP/Zimpel-2019/blob/master/Snps_pipeline)

USDA-VS pipeline used by Reis and Cunha:
[vSNP3](https://github.com/USDA-VS/vSNP3)
This is a conda python pipeline that "generates BAM, VCF and annotated SNP tables and corresponding phylogenetic trees to achieve a high resolution SNP analysis" [user manual](https://github.com/USDA-VS/vSNP/blob/master/docs/detailed_usage.md)

Irilenia's pipeline from 'omics practical
[genomeComparison_practical](file:///Users/jenniferstiens/Documents/omics_demo/GenomeComparison_practical_IN.html)
Uses freebayes to create VCF file

Zwyer pipeline using kvarQ
[kvarq testsuite](https://github.com/gagneux-lab/LivestockAssociatedMTBC/blob/main/KvarQ_testsuite/MTBC_animals/phylo.py)



### Basic steps
- Trim reads, remove duplicates and check quality (with fastp)
- Map to reference genome (*with bwa-mem)
- Picard to remove duplicates in mapped reads??
- find snps and generate vcf file (with samtools mpileup/varscan, or freebayes/vcftools)
- create consensus file and annotate (vcflib(vcf2fastq)/DFAST or snpEFF)
- remove repetitive genes and indels from analysis?
- compare to snp markers to determine lineage


### Facts to keep in mind:

SNP mutations are rare in bacterial species (typically around 1 mutation per 1 billion replications)

In [ ]:
# download files from AWS (downloaded list of links from microbesNG)
wget -i ~/myco_projects/WGS_variants_bovis/links_microbesng-data_amazonaws.txt

# change sample names to begin with 'dna'
cd fastqs
for FILENAME in *; do mv $FILENAME dna_$FILENAME; done

In [ ]:
# run fastp to trim, remove duplicates and check quality
# use snakefile for fastp

cd /Volumes/Data_disk/Mbovis_wgs
conda activate snakemake
snakemake -np -s ~/snakemake/fastp/pe/snakefile.smk
nohup snakemake --cores 2 -s ~/snakemake/fastp/pe/snakefile.smk > nohup.out 2>&1 &

# run quality control with fastqc
nohup snakemake --cores 2 -s ~/snakemake/fastqc/pe/snakefile.smk > nohup.out 2>&1 &


In [ ]:
#still having trouble completing fastqc without timing out
#added profile (see https://www.embl.org/groups/bioinformatics-rome/blog/2022/03/snakemake-profile-2-reducing-command-line-options-with-profile/)

vi ~/snakemake/profile/config.yaml

---

cores: 2
latency-wait: 60
reason: True
show-failed-logs: True
keep-going: True
printshellcmds: True
rerun-incomplete: True
restart-times: 3

# will finish but very slow as continually restarting

In [ ]:
snakemake -np --profile ~/snakemake/profile/.  -s ~/snakemake/fastqc/pe/snakefile.smk
nohup snakemake --profile ~/snakemake/profile/.  -s ~/snakemake/fastqc/pe/snakefile.smk > nohup.out 2>&1 &
cd fastqc_outputs
multiqc .

All samples between 0.7-1.3 M reads, except for A34 which is 0.3M reads (each 1 and 2)

## Map to Reference Genome 
Mycobacterium tuberculosis variant bovis AF2122/97, NC_002945.4

In [ ]:
#first need to create index files (using bwa) for reference genome (in same directory)
bwa index ~/myco_projects/ref_seqs/Mbovis/Mbovis_AF212297.fasta

snakemake -np --profile ~/snakemake/profile/. -s ~/snakemake/map_bwa/pe/snakefile.smk
nohup snakemake --profile ~/snakemake/profile/. -s ~/snakemake/map_bwa/pe/snakefile.smk > nohup_map.out 2>&1 &

cd sorted_reads
multiqc .

All but sample 35 have > 1M reads mapped


From Zimpel et al Snps pipeline (https://github.com/LaPAM-USP/Zimpel-2019/blob/master/Snps_pipeline):

#check depth in RDs positions (developed by Robson Francisco de Souza):
\ls *.bam | parallel --citation -N1 -j5 "samtools depth {} | awk '\$2 >= #POSITION1NITIAL# && \$2 <= #POSITION2FINAL#{a+=\$3;n++}END{print \"{.}\t\",a/n}'"
This relies on coverage of base positions, so need to run bam coverage, or the like?


#Picard
for i in `ls *.bam`:
    do picard MarkDuplicates REMOVE_DUPLICATES=true INPUT=$i OUTPUT=$i.dupl.bam METRICS_FILE=$i.txt
done

samtools view input.bam.dupl.bam | wc -l



In [ ]:
# Create conda env or add to another env
conda create --name dna python=3.8
conda activate dna  #or conda activate snakemake
conda install -c bioconda picard varscan freebayes vcftools snpeff


for i in `ls *.bam`:
    do picard MarkDuplicates REMOVE_DUPLICATES=true INPUT=$i OUTPUT=$i.dupl.bam METRICS_FILE=$i.txt
done

#new syntax warning:
#MarkDuplicates -REMOVE_DUPLICATES true -INPUT dna_236574_A20.bam -OUTPUT dna_236574_A20.bam.dupl.bam -METRICS_FILE dna_236574_A20.bam.txt

#takes a few minutes for 10 samples, but not too bad. Create new snakemake file or add to mapping etc?

#count reads after removing dups
for i in `ls *.dupl.bam`; do samtools view ${i} | wc -l; done
# dna_236574_A20.bam.dupl.bam   1188622
#  dna_236575_A25.bam.dupl.bam  1668956
#  dna_236576_A26.bam.dupl.bam  1361006
#  dna_236577_A27.bam.dupl.bam  1234336
#  dna_236578_A29.bam.dupl.bam  1272475
#  dna_236579_A30.bam.dupl.bam  1353532
#  dna_236580_A31.bam.dupl.bam  1277324
#  dna_236581_A32.bam.dupl.bam  1452253
#  dna_236582_A34.bam.dupl.bam   592629
#  dna_236583_A35.bam.dupl.bam  1898698


In [ ]:
#Find snps
#Samtools mpileup

#mpileup has replaced pileup in samtools for both multiple and single samples

#-q = min mapping quality; -Q = min base quality
cd sorted_reads
for i in $(ls *.dupl.bam); do samtools mpileup -f /Users/jenniferstiens/myco_projects/ref_seqs/Mbovis/Mbovis_AF212297.fasta -q 20 -Q 20 "$i" > "$i".mpileup; done

#takes some minutes

#move .mpileup files to vcf_files directory
mv *.mpileup /Volumes/Data_disk/Mbovis_wgs/vcf_files/


# test one file to create vcf with freebayes and with varscan
#freebayes (uses fasta files)
cd sorted_reads
freebayes -f  /Users/jenniferstiens/myco_projects/ref_seqs/Mbovis/Mbovis_AF212297.fasta -p 1  dna_236574_A20.bam > dna_236574_A20.vcffile

#varscan (uses mpileup files)
cd /Volumes/Data_disk/Mbovis_wgs/vcf_files
varscan mpileup2cns dna_236574_A20.bam.dupl.bam.mpileup --min-coverage 7 --min-var-freq 0.1 --min-freq-for-hom 0.90 --variants --output-vcf 1 > dna_236574_A20.bam.dupl.bam.mpileup.varscan.vcf




Sample lines from freebayes vcf file:
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	unknown
NC_002945.4	389	.	A	C	2.26038e-15	.	AB=0;ABP=0;AC=0;AF=0;AN=1;AO=7;CIGAR=1X;DP=121;DPB=121;DPRA=0;EPP=10.7656;EPPR=4.56684;GTI=0;LEN=1;MEANALT=2;MQM=60;MQMR=60;NS=1;NUMALT=1;ODDS=734.955;PAIRED=0.714286;PAIREDR=0.654867;PAO=0;PQA=0;PQR=0;PRO=0;QA=117;QR=3685;RO=113;RPL=3;RPP=3.32051;RPPR=8.56389;RPR=4;RUN=1;SAF=4;SAP=3.32051;SAR=3;SRF=51;SRP=5.3355;SRR=62;TYPE=snp	GT:DP:AD:RO:QR:AO:QA:GL	0:121:113,7:113:3685:7:117:0,-321.113
NC_002945.4	835	.	A	T	0	.	AB=0;ABP=0;AC=0;AF=0;AN=1;AO=6;CIGAR=1X;DP=115;DPB=115;DPRA=0;EPP=3.0103;EPPR=11.7958;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=60;NS=1;NUMALT=1;ODDS=742.237;PAIRED=0.833333;PAIREDR=0.981651;PAO=0;PQA=0;PQR=0;PRO=0;QA=99;QR=3715;RO=109;RPL=2;RPP=4.45795;RPPR=3.50834;RPR=4;RUN=1;SAF=3;SAP=3.0103;SAR=3;SRF=47;SRP=7.4927;SRR=62;TYPE=snp	GT:DP:AD:RO:QR:AO:QA:GL	0:115:109,6:109:3715:6:99:0,-325.433
NC_002945.4	1057	.	A	G	3465.73	.	AB=0;ABP=0;AC=1;AF=1;AN=1;AO=110;CIGAR=1X;DP=111;DPB=111;DPRA=0;EPP=9.40627;EPPR=0;GTI=0;LEN=1;MEANALT=2;MQM=60;MQMR=0;NS=1;NUMALT=1;ODDS=798.014;PAIRED=1;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=3921;QR=0;RO=0;RPL=58;RPP=3.72096;RPPR=0;RPR=52;RUN=1;SAF=72;SAP=25.8305;SAR=38;SRF=0;SRP=0;SRR=0;TYPE=snp	GT:DP:AD:RO:QR:AO:QA:GL	1:111:0,110:0:0:110:3921:-352.938,0

no filtering as set. Can use vcf tools to filter out low quality variants:

[freebayes and vcftools](https://hbctraining.github.io/In-depth-NGS-Data-Analysis-Course/sessionVI/lessons/02_variant-calling.html)

[vcf tools](https://vcftools.github.io/man_latest.html)

NOt sure about difference in how tools work--freebayes is evidently not 'alignment-based' 

[freebayes github](https://github.com/freebayes/freebayes)

Get a lot more feedback on std output with varscan mpileup2cns 
"This command makes consensus calls (SNP/Indel/Reference) from a mpileup file based on user-defined parameters"
OPTIONS:
	--min-coverage	Minimum read depth at a position to make a call [8]
	--min-reads2	Minimum supporting reads at a position to call variants [2]
	--min-avg-qual	Minimum base quality at a position to count a read [15]
	--min-var-freq	Minimum variant allele frequency threshold [0.01]
	--min-freq-for-hom	Minimum frequency to call homozygote [0.75]
	--p-value	Default p-value threshold for calling variants [99e-02]
	--strand-filter	Ignore variants with >90% support on one strand [1]
	--output-vcf	If set to 1, outputs in VCF format
	--variants	Report only variant (SNP/indel) positions (mpileup2cns only) [0]



Example output on one file:

Only variants will be reported
Warning: No p-value threshold provided, so p-values will not be calculated
Min coverage:	7
Min reads2:	2
Min var freq:	0.1
Min avg qual:	15
P-value thresh:	0.01
Reading input from dna_236574_A20.bam.dupl.bam.mpileup
4336824 bases in pileup file
816 variant positions (739 SNP, 77 indel)
21 were failed by the strand-filter
795 variant positions reported (719 SNP, 76 indel)


lines from file:

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	Sample1
NC_002945.4	1057	.	A	G	.	PASS	ADP=84;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:84:84:0:84:100%:4.3483E-50:0:36:0:0:52:32
NC_002945.4	4480	.	T	C	.	PASS	ADP=72;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:72:72:0:72:100%:6.7558E-43:0:36:0:0:43:29

on varscan manual page:
[varscan manual](https://varscan.sourceforge.net/using-varscan.html#v2.3_pileup2snp)
Note, to save disk space and file I/O, you can redirect mpileup output directly to VarScan with a "pipe" command. For example: 
samtools mpileup -f reference.fasta myData.bam | java -jar VarScan.v2.2.jar pileup2snp


In [ ]:
#varscan

cd /Volumes/Data_disk/Mbovis_wgs/vcf_files

# loop through mpileup files
for file in *.bam.dupl.bam.mpileup; do varscan mpileup2cns ${file} --min-coverage 7 --min-var-freq 0.1 --min-freq-for-hom 0.90 --variants --output-vcf 1 > ${file}.varscan.vcf; done

# annotate variant files with snpEff

# find available databases
snpEff databases | grep -i mycobacterium_bovis_af2122_97

#Mycobacterium_bovis_af2122_97                               	Mycobacterium_bovis_af2122_97                               	          	                              	
#[https://snpeff.blob.core.windows.net/databases/v5_1/snpEff_v5_1_Mycobacterium_bovis_af2122_97.zip, 
#https://snpeff.blob.core.windows.net/databases/v5_0/snpEff_v5_0_Mycobacterium_bovis_af2122_97.zip]
#Mycobacterium_bovis_af2122_97_gca_000195835                 	Mycobacterium_bovis_af2122_97_gca_000195835
# when I looked at the second in ncbi, it said 'assembly has been supressed'
#https://www.ncbi.nlm.nih.gov/assembly/GCF_000195835.2/
#                 	          	                              	[https://snpeff.blob.core.windows.net/databases/v5_1/snpEff_v5_1_Mycobacterium_bovis_af2122_97_gca_000195835.zip, https://snpeff.blob.core.windows.net/databases/v5_0/snpEff_v5_0_Mycobacterium_bovis_af2122_97_gca_000195835.zip]

cd /Volumes/Data_disk/Mbovis_wgs/vcf_files/
snpEff Mycobacterium_bovis_af2122_97 dna_236574_A20.bam.dupl.bam.mpileup.varscan.vcf > ann_vcf/dna_236574_A20.ann.vcf

#00:00:00 ERROR while connecting to https://snpeff.blob.core.windows.net/databases/v5_1/snpEff_v5_1_Mycobacterium_bovis_af2122_97.zip
# all say ERROR_CHROMOSOME_NOT_FOUND

#replace Chromosome name in vcf files so it can be used in snpEff
cd mbovis_vcf_files
for f in $(ls *.vcf)
do
    sed -i '' 's/NC_002945.4/Chromosome/g' "$f" > "$i".vcf  #extra empty string for mac
        echo "Processing $f"
done

snpEff -v -no-downstream -no-upstream Mycobacterium_bovis_af2122_97 mbovis_vcf_files/dna_236574_A20.bam.dupl.bam.mpileup.varscan.vcf > mbovis_vcf_files/test.annot.vcf

#use snakefile
#need to change names of files to match more generic for snakefile
for file in *
do
base=`basename $file .bam.dupl.bam`
mv $file $base.dupl.bam
done

# do this as go along in snakefile 
#snakemake -np --profile ~/snakemake/profile/. -s ~/snakemake/variant_calling/snakefile.smk
#nohup snakemake --profile ~/snakemake/profile/. -s ~/snakemake/variant_calling/snakefile.smk > nohup.out 2>&1 &

#this didn't work ERROR 'Does not match genome' and first base is always 'N', problem starts with mpileup

## Re map to H37Rv (do not do)

In zimpel paper, used H37Rv since bovis is missing no genes from Mtb. But this means i need to use H37Rv for mpileup. AND re-map using Mtb instead of Mbovis. **Correction, they used mbovis for snpEFF, see github https://github.com/LaPAM-USP/Zimpel-2019/blob/master/Snps_pipeline

In [ ]:
#redo all mapping/mpileup/varscan with H37Rv. 

#Re-mapped with mapping snakefile. Added picard de-duplicating to snakefile

snakemake -np --profile ~/snakemake/profile/. -s ~/snakemake/map_bwa/pe/snakefile.smk
nohup snakemake --profile ~/snakemake/profile/. -s ~/snakemake/map_bwa/pe/snakefile.smk > nohup_map.out 2>&1 &


#make new snakefile and use for mpileup and varscan (and eventually snpeff)
#snakemake/variant_calling/snakefile.smk

snakemake -np --profile ~/snakemake/profile/. -s ~/snakemake/variant_calling/snakefile.smk
nohup snakemake --profile ~/snakemake/profile/. -s ~/snakemake/variant_calling/snakefile.smk > nohup.out 2>&1 &

#getting errors trying to run samtools mpileup with the H37Rv reference. re index with samtools?
samtools mpileup -f /Users/jenniferstiens/myco_projects/ref_seqs/Mtb/AL123456.3.fasta -q 20 -Q 20 deduped_sorted_reads/dna_236574_A20.dupl.bam > dna_236574_A20.mpileup
samtools faidx ~/myco_projects/ref_seqs/Mtb/AL123456.3.fasta
# this did the trick. need different indexes for bwa and for samtools mpileup?

#snpEFF

#check Chromosome name (this was probably the same issue above with Mbovis mapped?)
awk '{print$1}' myfile.vcf | tail -n 1
#AL123456.3

#replace Chromosome name in vcf files so it can be used in snpEff
for f in $(ls *.vcf)
do
    sed -i 's/AL123456.3/Chromosome/g' "$f" > "$i".vcf
        echo "Processing $f"
done

# do do one at a time with pipe:
sed 's/AL123456.3/Chromosome/g' mtb_vcf_files/dna_236575_A25.varscan.vcf | snpEff -v -no-downstream -no-upstream Mycobacterium_tuberculosis_h37rv > test.annot.vcf

Assume these are different strains rather than different samples from same strain and run each separately. Want to spoligotype the genomes as well. there are in silico measures to find spoligotype patterns https://github.com/kvarq/kvarq and https://github.com/xiaeryu/SpoTyping-v2.0. Then you can use these patterns in Mbovis spoligotype database https://www.mbovis.org to assign names to spoligotypes. (look at Zwyer paper)

Downloaded spreadsheet of SNPs and associated lineages from Zimpel et al, 2020. Maybe can use this to compare position and allele and identify common snps with my WGS data?

https://github.com/gagneux-lab/LivestockAssociatedMTBC/blob/main/KvarQ_testsuite/MTBC_animals/phylo.py

From this paper: https://open-research-europe.ec.europa.eu/articles/1-100

"In the cases for which WGS exist, sequencing reads can be queried with a new suite of markers (See Zenodo reposi- tory), using KvarQ and bypassing the need to run conventional alignment approaches and phylogenetic analysis for strain classification.”


Can't install kvarq on my laptop--Dave H has installed on thoth server


Downloaded spreadsheet of SNPs, could do manually if kvarq doesn't work.

In [ ]:
#to use kvarq on thoth
#don't log into bash shell or activate conda env since need different python version loaded

module use -a /s/software/modules
module load python/v2
conda activate kvarq
kvarq --help

## kvarq on thoth

Download sequences (fastq) from aws to thoth using wget

In [ ]:
#transfer list of aws links
scp ~/myco_projects/WGS_variants_bovis/links_microbesng-data_amazonaws.txt sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/mbovis_wgs/fastq
cd /d/in16/u/sj003/mbovis_wgs/fastq
# download files from AWS (downloaded list of links from microbesNG)
wget -i links_microbesng-data_amazonaws.txt

#move testsuite from zwyer to new testsuites directory (all files need to be there)
mkdir testsuites
scp ~/PycharmProjects/phylogeny/_util.py sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/mbovis_wgs/testsuites
scp ~/PycharmProjects/phylogeny/phylo.py sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/mbovis_wgs/testsuites
scp ~/Desktop/MTB_ancestor_reference.bases sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/mbovis_wgs/testsuites/


#scan file with kvarq (creates .json coverages file)
kvarq scan -l MTBC -p fastq/dna_236574_A20_1.fastq.gz dna_236574_A20_1.json

#load testsuite *must just indicate working directory and then name of path separately
kvarq -t . info -l testsuites/phylo

#check gui
kvarq gui
# testsuite doesn't show up on gui, restricted to command line

#scan with zwyer testsuite
kvarq -t . scan -l testsuites/phylo fastq/dna_236574_A20_1.fastq.gz dna_236574_A20_1.json

#make loop to scan each file
# run_kvarq.sh
#!/bin/bash
for file in fastq/*
do
        base=`basename $file .fastq.gz`
        kvarq -t . scan -l testsuites/phylo $file $base.json
done

chmod +xr run_kvarq.sh
bash run_kvarq.sh

# takes a few minutes for each, may want to run in background in future

For these files: 
fastq/dna_236579_A30_1.fastq.gz, fastq/dna_236579_A30_2.fastq.gz 
fastq/dna_236581_A32_1.fastq.gz, fastq/dna_236581_A32_2.fastq.gz

got the following error:
    "cannot read next deflated stream in compressed file : expected method==DEFLATED"
    "could not scan fastq/dna_236581_A32_2.fastq.gz : error while inflating compressed data"

Still produced .json files for A30, but not A32

Decompressed file ahead of time and then run again

In [ ]:
gunzip fastq/dna_236581_A32_1.fastq.gz fastq/dna_236581_A32_2.fastq.gz
kvarq -t . scan -l testsuites/phylo fastq/dna_236581_A32_1.fastq fastq/dna_236581_A32_1.json
kvarq -t . scan -l testsuites/phylo fastq/dna_236581_A32_2.fastq fastq/dna_236581_A32_2.json

Makes a coverage file and separate analysis for each one of each pair. 
Parsed json files to get lineage assignment in R 'mbovis_wgs.Rmd'

fastq/dna_236574_A20_1.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236574_A20_2.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236575_A25_1.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236575_A25_2.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236576_A26_1.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236576_A26_2.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236577_A27_1.fastq.gz	La1/La1.7-La1.7.1 -- mixed coverage
fastq/dna_236577_A27_2.fastq.gz	La1/La1.7-La1.7.1 -- mixed coverage
fastq/dna_236578_A29_1.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236578_A29_2.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236579_A30_1.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236579_A30_2.fastq.gz	La1/La1.8-La1.8.2
fastq/dna_236580_A31_1.fastq.gz	La1/La1.2 -- mixed coverage
fastq/dna_236580_A31_2.fastq.gz	La1/La1.2 -- mixed coverage
fastq/dna_236581_A32_1.fastq	La1/La1.8-La1.8.2 -- mixed coverage
fastq/dna_236581_A32_2.fastq	La1/La1.8-La1.8.2 -- mixed coverage
fastq/dna_236582_A34_1.fastq.gz	La1/La1.7-La1.7.1 -- mixed coverage
fastq/dna_236582_A34_2.fastq.gz	La1/La1.7 -- low coverage (median below 10x) -- mixed coverage
fastq/dna_236583_A35_1.fastq.gz	La1/La1.7-La1.7.1 -- mixed coverage
fastq/dna_236583_A35_2.fastq.gz	La1/La1.7-La1.7.1 -- mixed coverage

In [ ]:
library(here)
library(tidyverse)
library(jsonlite)

file <- read_json(here("coverage_files/dna_236574_A20_1.json"), simplifyVector = T)
#file$analyses$`testsuites/phylo` #gives result of testsuites/phylo 
#file$info$fastq #fastq gives name
#file$coverages #coverage of each snp

cov_files <- dir(here("coverage_files"), pattern="*.json")
#data <- cov_files %>%
#       map_df(~fromJSON(file.path(here("coverage_files"), .), flatten = TRUE))

kvarq_df <- NULL
kvarq_df <- as.data.frame(matrix("", ncol=2))
colnames(kvarq_df) <- c("file", "lineage")

filenames <- NULL
lineages <- NULL

for (i in 1:length(cov_files)){
  fasta <- read_json(here("coverage_files", cov_files[i]), simplifyVector = T)
  filenames <- c(filenames, fasta$info$fastq[[1]])
  lineages  <- c(lineages, fasta$analyses$`testsuites/phylo`)
}
kvarq_df <- tibble(samples=filenames, analysis=lineages)

# simplify sample names
kvarq_df$samples <- sub("fastq/dna_", "", kvarq_df$samples)

kvarq_df$samples <- sub(".fastq.gz", "", kvarq_df$samples)
kvarq_df$samples <- sub(".fastq", "", kvarq_df$samples)
                        

# Split name column into firstname and last name
kvarq_df[c('Primary_lineage', 'Secondary_lineage')] <- str_split_fixed(kvarq_df$analysis, '/', 2)
kvarq_df[c('Secondary_lineage', 'Tertiary_lineage')] <- str_split_fixed(kvarq_df$Secondary_lineage, '-', 2)
View(kvarq_df)

#write_tsv(kvarq_df, here("mbovis_lineage.tsv"), col_names=T)

Most of the samples from lineage 1.8 (1.8.2 specifically)
1.8.2 is subdivided from Eu1 clonal complex. INcludes isolates from France and continental Europe, Africa, including Madagascar.

3 samples are 1.7.1, though these had more 'mixed coverage'. 1.7.1 is an Eu2 clonal complex subgroup within 1.7. Occurs in Western Europe, America and Southern Africa. Few genomes known

(lineage info from Zwyer et al, 2021)

## vcf parsing

Look at vcf files from each sample to identify genes with snps and degree of predicted impact

In [ ]:
library(tidyverse)

data <- read_tsv(here("mbovis_vcf_files/dna_236574_A20.annot.vcf"), comment = "##", col_names=TRUE)

 #INFO_cols <- c("Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO' ">
##INFO=<ID=LOF,Number=.,Type=String,Description="Predicted loss of function effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">
##INFO=<ID=NMD,Number=.,Type=String,Description="Predicted nonsense mediated decay effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">

a <- str_split(data$INFO[1], "\\|")
a
length(data$INFO)
info_data <- str_split(data$INFO, "\\|")
ncol_info <- length(info_data[[1]])
head(info_data)
my_cols <- c("Allele", "Annotation","Annotation_Impact","Gene_Name","Gene_ID","Feature_Type","Feature_ID", "Transcript_BioType" ,"Rank","HGVS.c","HGVS.p","cDNA.pos/cDNA.length","CDS.pos/CDS.length","AA.pos_AA.length","Distance","errors/warnings/info")
vcf_data <- as_tibble(lapply(1:ncol_info,function(i)sapply(info_data,"[",i)), .name_repair="minimal")
colnames(vcf_data) <- my_cols

#Look at particular categories of mutations
vcf_data %>% filter(Annotation_Impact=="HIGH")

#save data as spreadsheet
#write_excel_csv(vcf_data, here("mbovis_vcf_files/vcf_snpeff_results.csv"))

filenames <- dir(here("mbovis_vcf_files"), pattern="*.annot.vcf")
for (i in 1:length(filenames)){
  sample <- substr(filenames[i], 1, 14)
  data <- read_tsv(here("mbovis_vcf_files", filenames[i]), comment = "##",
                   col_names=TRUE, show_col_types = F)
  info_data <- str_split(data$INFO, "\\|")
  vcf_data <- as_tibble(lapply(1:ncol_info,function(i)sapply(info_data,"[",i)),
                        .name_repair="minimal")
  colnames(vcf_data) <- my_cols
  res_filename <- paste(sample, "snpEff_res.csv", sep="_")
  write_excel_csv(vcf_data, here("mbovis_vcf_files/vcf_snpeff_res", res_filename))
}


Make nested dataframe of all results from all samples.

In [ ]:
library(tidyverse)
library(purrr)
# all in one nested dataframe with file information

data_path <- here("mbovis_vcf_files/vcf_snpeff_res")

filenames <- dir(data_path, pattern="*.csv")
df1 <- data_frame(filename=filenames) %>%
  mutate(file_contents = map(filenames,          # read files into
           ~ read_csv(file.path(data_path, .), col_names = T, show_col_types = F)) # a new data column
        )  
colnames(unnest(df1, cols=2))
# rename with sample name instead of filename 
df1[[1]]<-sub("_snpEff_res.csv", "", df1[[1]])

# list of nested dfs
unnest(df1, cols = 1)

# all unnested
all_data <- unnest(df1, cols=2)

# get particular sample
all_data %>%
  filter(filename=="dna_236574_A20")

# all mutations of particular gene
all_data %>%
  filter(Gene_Name=="pks3") %>%
  group_by(filename) 

#save data 
#saveRDS(all_data, here("R_data/snpEff_annot_vcf.RData"))

## look for snps in phoP and phoR and intergenic regions adjacent

snp_data %>% filter(grepl("Mb0780", Gene_ID))

No snps in phoP and phoR or adjacent intergenic regions.